In [1]:
import geopandas as gpd
import pandas as pd
import os

def load_geojson_data(year, base_path):
    folder_name = f"{year}"
    file_path = os.path.join(base_path, folder_name, "hex_total_cost.geojson")
    
    # Read the GeoJSON file
    gdf = gpd.read_file(file_path)
    
    # Convert GeoDataFrame to DataFrame with relevant columns
    # Ensure to adjust this to fit your actual GeoDataFrame structure
    df = pd.DataFrame({
        'Year': year,
        'Scenario': folder_name,  # Assuming folder name represents the scenario
        'Production Cost': gdf['Vientiane trucking production cost'],  # Replace with actual column name
        'Geometry': gdf.geometry.apply(lambda geom: geom.wkt)  # Convert geometry to WKT string
    })
    
    return df


In [2]:
def preprocess_all_data(base_path, years):
    all_data = []
    
    for year in years:
        df = load_geojson_data(year, base_path)
        all_data.append(df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df


In [3]:
base_path = r"C:\Users\lukas\OneDrive - Imperial College London\0_Thesis\0relevant_repositories\GeoH2 Case Study Laos\Resources\0Temporal\ratio_total"
years = ["2022", "2025", "2030", "post_2030"]

# Preprocess all data
df = preprocess_all_data(base_path, years)


In [19]:
df = preprocess_all_data(base_path, years)
df

,Year,Production Cost,Geometry
0,2022,11.702678,"{""type"": ""Polygon"", ""coordinates"": [[[105.7754..."
1,2022,11.119620,"{""type"": ""Polygon"", ""coordinates"": [[[101.8970..."
2,2022,10.944596,"{""type"": ""Polygon"", ""coordinates"": [[[103.4496..."
3,2022,11.222974,"{""type"": ""Polygon"", ""coordinates"": [[[103.0391..."
4,2022,11.367160,"{""type"": ""Polygon"", ""coordinates"": [[[102.8738..."
...,...,...,...
3294,post_2030,7.649760,"{""type"": ""Polygon"", ""coordinates"": [[[101.1425..."
3295,post_2030,9.285439,"{""type"": ""Polygon"", ""coordinates"": [[[105.9556..."
3296,post_2030,9.285439,"{""type"": ""Polygon"", ""coordinates"": [[[105.9556..."
3297,post_2030,8.321759,"{""type"": ""Polygon"", ""coordinates"": [[[105.1541..."


In [21]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import json
import os
import geopandas as gpd
from shapely.geometry import mapping

# Define base path and years
base_path = r"C:\Users\lukas\OneDrive - Imperial College London\0_Thesis\0relevant_repositories\GeoH2 Case Study Laos\Resources\0Temporal\ratio_total"
years = ["2022", "2025", "2030", "post_2030"]

# Function to load and preprocess GeoJSON data
def load_geojson_data(year, base_path):
    file_path = os.path.join(base_path, year, "hex_total_cost.geojson")
    
    # Read the GeoJSON file using GeoPandas
    gdf = gpd.read_file(file_path)
    
    # Ensure the geometry is present and convert it to GeoJSON format
    if 'geometry' not in gdf.columns:
        raise ValueError(f"No geometry column found in {file_path}")

    # Drop rows where 'Vientiane trucking production cost' is null
    gdf = gdf.dropna(subset=['Vientiane trucking production cost'])
    
    # Convert GeoDataFrame to DataFrame with relevant columns
    df = pd.DataFrame({
        'Year': year,
        'Production Cost': gdf['Vientiane trucking production cost'],
        'Geometry': gdf['geometry'].apply(lambda geom: json.dumps(mapping(geom)))  # Convert geometry to GeoJSON format
    })
    
    return df

# Function to preprocess data from all years
def preprocess_all_data(base_path, years):
    all_data = []
    for year in years:
        df = load_geojson_data(year, base_path)
        all_data.append(df)
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

# Preprocess all data
df = preprocess_all_data(base_path, years)

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Hexagon Visualization'),

    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(df['Year'].unique())],
        value=sorted(df['Year'].unique())[0]  # Default value
    ),

    dcc.Graph(id='hexagon-map')
])

@app.callback(
    Output('hexagon-map', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_figure(selected_year):
    filtered_df = df[df['Year'] == selected_year]
    
    if filtered_df.empty:
        return px.scatter_mapbox()  # Return empty figure if no data available

    # Convert the Geometry column from JSON strings to Python dicts
    filtered_df['Geometry'] = filtered_df['Geometry'].apply(json.loads)
    
    # Create a choropleth mapbox using GeoJSON geometry
    fig = px.choropleth_mapbox(
        filtered_df,
        geojson=filtered_df['Geometry'].tolist(),  # Convert to list of dicts
        locations=filtered_df.index,  # Use index or a unique identifier column for locations
        color='Production Cost',
        color_continuous_scale="Viridis",
        range_color=[filtered_df['Production Cost'].min(), filtered_df['Production Cost'].max()],
        mapbox_style="carto-positron",
        center={"lat": 18.5, "lon": 105.0},  # Center around Laos
        zoom=5,  # Adjust zoom level
        title=f'Production Cost in {selected_year}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\lukas\AppData\Local\Temp\ipykernel_6556\4152802073.py:76: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
import geopandas as gpd
import plotly.express as px
import pandas as pd
import json
from shapely.geometry import mapping
import plotly.io as pio
import webbrowser

# Define the path to your GeoJSON file for the year 2022
file_path = r"C:\Users\lukas\OneDrive - Imperial College London\0_Thesis\0relevant_repositories\GeoH2 Case Study Laos\Resources\0Temporal\ratio_total\2022\hex_total_cost.geojson"

# Load the GeoJSON file using GeoPandas
gdf = gpd.read_file(file_path)

# Display the first few rows of the GeoDataFrame to understand its structure
print(gdf.head())

# Ensure that 'Vientiane trucking production cost' is a valid column
if 'Vientiane trucking production cost' not in gdf.columns:
    raise ValueError("No 'Vientiane trucking production cost' column found in the GeoDataFrame")

# Drop rows where 'Vientiane trucking production cost' is null
gdf = gdf.dropna(subset=['Vientiane trucking production cost'])

# Convert geometries to GeoJSON format using shapely's mapping function
gdf['Geometry'] = gdf['geometry'].apply(lambda geom: mapping(geom))

# Construct a DataFrame with relevant columns
df = pd.DataFrame({
    'Production Cost': gdf['Vientiane trucking production cost'],
    'Geometry': gdf['Geometry']
})

# Convert DataFrame to a list of features for the GeoJSON
features = []
for i, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": row['Geometry'],
        "properties": {
            "Production Cost": row['Production Cost']
        }
    }
    features.append(feature)

# Construct GeoJSON FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Create a choropleth mapbox using GeoJSON geometry
fig = px.choropleth_mapbox(
    df,
    geojson=geojson_data,
    locations=df.index,
    color='Production Cost',
    color_continuous_scale="Viridis",
    range_color=[df['Production Cost'].min(), df['Production Cost'].max()],
    mapbox_style="carto-positron",
    center={"lat": 18.5, "lon": 105.0},  # Center around Laos
    zoom=5,  # Adjust zoom level
    title='Production Cost in 2022'
)

# Show the figure
# Option 1: Use Plotly default rendering
try:
    fig.show()
except ValueError as e:
    print(f"Error with fig.show(): {e}")
    # Option 2: Save as HTML and open in browser
    pio.write_html(fig, 'figure.html')
    webbrowser.open('figure.html')


          h3_index   n0   n1   n2   n3   n4   n5  waterbody_dist  \
0  85416c0bfffffff  662   61  296  270  736  152        0.000000   
1  85414d3bfffffff  753  187  514  281  195  647        0.000000   
2  856496cffffffff  677  118  263  633  511  937       11.131949   
3  856494c7fffffff  553  920  790  146  660  531       22.263898   
4  8541486bfffffff  484  576   21  935  866    9        0.000000   

   waterway_dist  road_dist  ...  Vientiane pipeline battery capacity  \
0            0.0        0.0  ...                                  0.0   
1            0.0        0.0  ...                                  0.0   
2            0.0        0.0  ...                                  0.0   
3            0.0        0.0  ...                                  0.0   
4            0.0        0.0  ...                                  0.0   

   Vientiane pipeline H2 storage capacity  Vientiane pipeline production cost  \
0                             3316.055178                              